## ❤ 모듈 불러오기

In [1]:
import os, re 
import numpy as np
import tensorflow as tf

## 🧡 데이터 읽어오기
- glob 모듈을 사용하면 파일을 읽어오는 작업을 하기가 아주 용이해요. glob 를 활용하여 모든 txt 파일을 읽어온 후, raw_corpus 리스트에 문장 단위로 저장하도록 할게요!

In [2]:
import glob
import os

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담습니다.
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 💛 데이터 정제
- preprocess_sentence() 함수를 만든 것을 기억하시죠? 이를 활용해 데이터를 정제하도록 하겠습니다.

- 추가로 지나치게 긴 문장은 다른 데이터들이 과도한 Padding을 갖게 하므로 제거합니다. 너무 긴 문장은 노래 가사 작사하기에 어울리지 않을 수도 있겠죠.
그래서 이번에는 문장을 토큰화 했을 때 토큰의 개수가 15개를 넘어가는 문장을 학습 데이터에서 제외하기를 권합니다.

In [3]:
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

# 이 문장이 어떻게 필터링되는지 확인해 보세요.
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
# 여기에 정제된 문장을 모을겁니다
corpus = []

for sentence in raw_corpus:
    # 우리가 원하지 않는 문장은 건너뜁니다
    if (len(sentence) == 0): continue
    if sentence[-1] == ":": continue
    
    # 정제를 하고 담아주세요
    preprocessed_sentence = preprocess_sentence(sentence)
    corpus.append(preprocessed_sentence)

print(f'총 데이터 수 : {len(corpus)}')

# 정제된 결과를 10개만 확인해보죠
corpus[:10]

총 데이터 수 : 175749


['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [5]:
len(corpus[0].split()) # 0번째 문장의 단어 개수

11

## 💚 평가 데이터셋 분리
- 훈련 데이터와 평가 데이터를 분리하세요!

- tokenize() 함수로 데이터를 Tensor로 변환한 후, sklearn 모듈의 train_test_split() 함수를 사용해 훈련 데이터와 평가 데이터를 분리하도록 하겠습니다. 단어장의 크기는 12,000 이상 으로 설정하세요! 총 데이터의 20% 를 평가 데이터셋으로 사용해 주세요!

In [6]:
def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer를 만들겁니다
    # 우리는 이미 문장을 정제했으니 filters가 필요없어요
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 바꿀거에요
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000,
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성합니다
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환합니다
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 입력 데이터의 시퀀스 길이를 일정하게 맞춰줍니다
    # 만약 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춰줍니다.
    # 토큰수를 15개 이하로 제한합니다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post',maxlen=16)
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    5 ...    0    0    0]
 [   2   17 2639 ...    0    0    0]
 [   2   36    7 ...    3    0    0]
 ...
 [ 130    5   22 ...   10 1013    3]
 [   5   37   15 ...  877  647    3]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f4b6dce7af0>


In [7]:
print(tensor[:3, :])

[[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
     0    0]
 [   2   17 2639  873    4    8   11 6043    6  329    3    0    0    0
     0    0]
 [   2   36    7   37   15  164  282   28  299    4   47    7   43    3
     0    0]]


In [8]:
for idx in tokenizer.index_word: # 단어 확인
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [9]:
# tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성합니다
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성합니다.
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    5   91  297   65   57    9  969 6042    3    0    0    0
    0]
[  50    5   91  297   65   57    9  969 6042    3    0    0    0    0
    0]


In [10]:
len(src_input),len(tgt_input) # 데이터 확인

(175749, 175749)

In [11]:
# 데이터 분리
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(
                                                    src_input,
                                                    tgt_input,
                                                    test_size=0.2, 
                                                    shuffle= True,
                                                    random_state=32)

In [12]:
enc_train.shape, enc_val.shape, dec_train.shape, dec_val.shape # 훈련데이터, 테스트데이터 모양


((140599, 15), (35150, 15), (140599, 15), (35150, 15))

## 💙 인공지능 만들기
- 모델의 Embedding Size와 Hidden Size를 조절하며 10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!

- 잘 설계한 모델을 학습하려면, model.fit() 함수를 사용해야 합니다. model.fit() 함수에는 다양한 인자를 넣어주어야 하는데, 가장 기본적인 인자로는 데이터셋과 epochs가 있습니다. '5. 실습 (2) 인공지능 학습시키기'에서의 예시와 같이 말이죠.

- model.fit(dataset, epochs=30)
- 하지만 model.fit() 함수의 epochs를 아무리 크게 넣는다 해도 val_loss 값은 2.2 아래로 떨어지지 않습니다. 이럴 경우는 batch size를 변경하는 것과 같이 model.fit() 함수에 다양한 인자를 넣어주면 해결될 수도 있습니다. 자세한 내용은 https://www.tensorflow.org/api_docs/python/tf/keras/Model#fit 를 참고하세요!

In [13]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 2000
hidden_size = 1900
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [14]:
optimizer = tf.keras.optimizers.Adam()
#Loss
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')
0
model.compile(loss=loss, optimizer=optimizer)

model.fit(enc_train,dec_train, epochs=4,validation_data=(enc_val, dec_val))

Epoch 1/4
4394/4394 [==============================] - 574s 127ms/step - loss: 2.8718 - val_loss: 2.5794
Epoch 2/4
4394/4394 [==============================] - 563s 128ms/step - loss: 2.3019 - val_loss: 2.3430
Epoch 3/4
4394/4394 [==============================] - 563s 128ms/step - loss: 1.8792 - val_loss: 2.2310
Epoch 4/4
4394/4394 [==============================] - 564s 128ms/step - loss: 1.5614 - val_loss: 2.1991


In [15]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  24002000  
_________________________________________________________________
lstm (LSTM)                  multiple                  29647600  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  28887600  
_________________________________________________________________
dense (Dense)                multiple                  22813901  
Total params: 105,351,101
Trainable params: 105,351,101
Non-trainable params: 0
_________________________________________________________________


## 💜 문장생성 

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환합니다
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 만듭니다
    #    1. 입력받은 문장의 텐서를 입력합니다
    #    2. 예측된 값 중 가장 높은 확률인 word index를 뽑아냅니다
    #    3. 2에서 예측된 word index를 문장 뒤에 붙입니다
    #    4. 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성을 마칩니다
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환합니다 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [17]:
generate_text(model, tokenizer, init_sentence="<start> i love", max_len=20)

'<start> i love the way you lie shine bright like a diamond <end> '

## 🤎 회고

### 전체과정 정리
1. 데이터 정제
    - 정규표현식 사용
    - 전처리
2. 텐서 변환
    - 데이터를 토큰화 하고 텐서로 변환 
    - 이과정에서 maxlen을 사용하여 최대 토큰개수를 15개 이하로 제한하였다.
3. 데이터 분리 
    - 학습과 테스트를 위해 데이터를 분리해 주었다.
4. 모델 학습 
    - 하이퍼 파리미터 조정을 통해 val_loss가 2.2 이하로 나오게 설정
5. 문장생성
    - 학습한 결과를 토대로 실제 문장을 생성해 보았다.
    
### 고난과 역경

- val_loss를 줄이는 과정에서 상당한 시련을 맛봤다. batch_size, embedding_size, hidden_size등 다양한 파라미터를 조정해 보았고 그결과 2,19라는 값진 결과를 얻을 수 있었다.

- 문장 출력결과 해석하기 애매하긴 하지만 문장을 잘 형성 하는것으로 보인다.
